# PyPersistent Demo - High-Performance Immutable Data Structures

This notebook demonstrates the key features of **pypersistent**, a high-performance library of persistent (immutable) data structures for Python, written in C++.

[![GitHub](https://img.shields.io/badge/GitHub-pypersistent-blue)](https://github.com/cmarschner/pypersistent)
[![PyPI](https://img.shields.io/pypi/v/pypersistent)](https://pypi.org/project/pypersistent/)

## Installation

First, let's install pypersistent:

In [ ]:
!pip install pypersistent

## What are Persistent Data Structures?

**Persistent data structures** are immutable - every modification creates a new version while keeping the old one intact. They use **structural sharing** to make this efficient:

```python
# Python dict - must copy everything
d1 = {i: f'val{i}' for i in range(10000)}
d2 = d1.copy()  # Copies all 10,000 entries
d2['new_key'] = 'new_value'

# PersistentDict - shares structure
from pypersistent import PersistentDict
m1 = PersistentDict.from_dict({i: f'val{i}' for i in range(10000)})
m2 = m1.set('new_key', 'new_value')  # Shares 99.99% of structure with m1
```

**Key Benefits:**
- 🔒 **Thread-safe** by default (immutability = no locks needed)
- 📸 **Easy snapshots** for undo/redo, version history
- 🚀 **Fast updates** through structural sharing
- 🎯 **3-5x faster** than pure Python alternatives

## 1. PersistentDict - Immutable Hash Map

The most common data structure - an unordered key-value map based on HAMT (Hash Array Mapped Trie).

In [ ]:
from pypersistent import PersistentDict

# Create a dict
m1 = PersistentDict.create(name='Alice', age=30, city='NYC')
print("Original:", m1)

# Add a new key - creates a new version
m2 = m1.set('role', 'developer')
print("\nAfter adding 'role':", m2)
print("Original unchanged:", m1)  # m1 still the same!

# Access values
print("\nAccess:")
print(f"  m2['name'] = {m2['name']}")
print(f"  m2.get('salary', 'N/A') = {m2.get('salary', 'N/A')}")

# Check membership
print(f"\n  'role' in m2: {'role' in m2}")
print(f"  'role' in m1: {'role' in m1}")

### Pythonic Operations

In [ ]:
# Merge using | operator
m3 = m1 | {'salary': 100000, 'department': 'Engineering'}
print("Merged:", m3)

# Delete a key
m4 = m3.delete('age')
print("\nAfter deleting 'age':", m4)
print("m3 still has age:", m3.get('age'))

# Iteration
print("\nIterating over keys:")
for key in m2.keys():
    print(f"  {key}: {m2[key]}")

### Bulk Operations

In [ ]:
# Create from dictionary - fast bulk construction
data = {f'user_{i}': {'id': i, 'score': i * 10} for i in range(1000)}
users = PersistentDict.from_dict(data)

print(f"Created dict with {len(users)} users")
print(f"Sample: user_42 = {users['user_42']}")

# Get all items efficiently
items = users.items_list()[:5]  # First 5 items
print(f"\nFirst 5 items: {items}")

## 2. PersistentList - Immutable Vector

An indexed sequence with fast random access and efficient append.

In [ ]:
from pypersistent import PersistentList

# Create a list
v1 = PersistentList.create(1, 2, 3, 4, 5)
print("Original:", v1)

# Append (conj = conjoin)
v2 = v1.conj(6)
print("After append:", v2)
print("Original unchanged:", v1)

# Update by index
v3 = v2.assoc(0, 10)  # Change index 0 to 10
print("\nAfter updating index 0:", v3)
print("v2 unchanged:", v2)

# Random access
print(f"\nv3[3] = {v3[3]}")
print(f"v3.nth(4) = {v3.nth(4)}")

### List Operations

In [ ]:
# Slicing
v4 = PersistentList.from_list(list(range(10)))
print("Original:", v4)
print("v4[2:7]:", v4[2:7].list())
print("v4[:5]:", v4[:5].list())

# Remove last element
v5 = v4.pop()
print("\nAfter pop():", v5)

# Convert to Python list
python_list = v4.list()
print("\nAs Python list:", python_list)

## 3. PersistentSet - Immutable Set

An unordered collection of unique elements.

In [ ]:
from pypersistent import PersistentSet

# Create a set
s1 = PersistentSet.create(1, 2, 3, 4, 5)
print("Set 1:", s1)

# Add element
s2 = s1.conj(6)
print("After adding 6:", s2)

# Remove element
s3 = s2.disj(3)  # disj = disjoin
print("After removing 3:", s3)

# Membership
print(f"\n4 in s3: {4 in s3}")
print(f"3 in s3: {3 in s3}")

### Set Operations

In [ ]:
s4 = PersistentSet.create(3, 4, 5, 6, 7)
s5 = PersistentSet.create(5, 6, 7, 8, 9)

print("s4:", s4)
print("s5:", s5)

# Union
print("\nUnion (s4 | s5):", s4 | s5)

# Intersection
print("Intersection (s4 & s5):", s4 & s5)

# Difference
print("Difference (s4 - s5):", s4 - s5)

# Symmetric difference
print("Symmetric diff (s4 ^ s5):", s4 ^ s5)

## 4. PersistentSortedDict - Sorted Map

A key-value map that maintains keys in sorted order, based on Red-Black Tree.

In [ ]:
from pypersistent import PersistentSortedDict

# Create a sorted dict (keys automatically sorted)
sm1 = PersistentSortedDict.from_dict({3: 'three', 1: 'one', 2: 'two', 5: 'five', 4: 'four'})
print("Keys (always sorted):", list(sm1.keys()))

# Add a key - maintains sort order
sm2 = sm1.assoc(0, 'zero')
print("After adding 0:", list(sm2.keys()))

# Iterate in order
print("\nOrdered iteration:")
for k, v in sm2.items():
    print(f"  {k}: {v}")

### Range Queries

In [ ]:
# Create a larger sorted dict
scores = PersistentSortedDict.from_dict({i: f'score_{i}' for i in range(0, 100, 10)})
print("All keys:", list(scores.keys()))

# Get range [20, 50)
subset = scores.subseq(start=20, end=50, start_inclusive=True, end_inclusive=False)
print("\nRange [20, 50):", list(subset.keys()))

# Get first and last
print("\nFirst entry:", scores.first())
print("Last entry:", scores.last())

## 5. Pickle Support

All data structures support Python's pickle protocol for serialization.

In [ ]:
import pickle

# Create a complex nested structure
data = PersistentDict.create(
    users=PersistentList.create('Alice', 'Bob', 'Charlie'),
    scores=PersistentDict.from_dict({'Alice': 95, 'Bob': 87, 'Charlie': 92}),
    tags=PersistentSet.create('python', 'immutable', 'fast')
)

print("Original:", data)

# Serialize
pickled = pickle.dumps(data)
print(f"\nPickled size: {len(pickled)} bytes")

# Deserialize
restored = pickle.loads(pickled)
print("\nRestored:", restored)
print("Equal:", data == restored)

## 6. Performance Comparison

Let's see the performance benefits of structural sharing.

In [ ]:
import time

# Test: Create 100 variants of a dict with 10,000 elements
SIZE = 10_000
VARIANTS = 100

# Python dict approach
print("Testing Python dict (with copying)...")
base_dict = {i: f'val{i}' for i in range(SIZE)}
start = time.time()
variants = []
for i in range(VARIANTS):
    d = base_dict.copy()
    d[f'new_{i}'] = f'value_{i}'
    variants.append(d)
dict_time = time.time() - start
print(f"  Time: {dict_time*1000:.2f}ms")

# PersistentDict approach
print("\nTesting PersistentDict (with structural sharing)...")
base_pdict = PersistentDict.from_dict({i: f'val{i}' for i in range(SIZE)})
start = time.time()
pvariants = []
for i in range(VARIANTS):
    pd = base_pdict.set(f'new_{i}', f'value_{i}')
    pvariants.append(pd)
pdict_time = time.time() - start
print(f"  Time: {pdict_time*1000:.2f}ms")

# Compare
speedup = dict_time / pdict_time
print(f"\n✨ PersistentDict is {speedup:.1f}x faster for creating {VARIANTS} variants!")

## 7. Real-World Use Case: Version History

Persistent data structures are perfect for implementing undo/redo functionality.

In [ ]:
class Document:
    def __init__(self):
        self.history = [PersistentDict.create(content='', title='Untitled')]
        self.current_index = 0
    
    @property
    def current(self):
        return self.history[self.current_index]
    
    def update(self, **changes):
        # Create new version from current
        new_version = self.current
        for key, value in changes.items():
            new_version = new_version.set(key, value)
        
        # Trim history after current point and add new version
        self.history = self.history[:self.current_index + 1]
        self.history.append(new_version)
        self.current_index += 1
    
    def undo(self):
        if self.current_index > 0:
            self.current_index -= 1
            return True
        return False
    
    def redo(self):
        if self.current_index < len(self.history) - 1:
            self.current_index += 1
            return True
        return False

# Demo
doc = Document()
print("Initial:", doc.current)

doc.update(title='My Document')
print("\nAfter setting title:", doc.current)

doc.update(content='Hello, World!')
print("After adding content:", doc.current)

doc.update(content='Hello, Persistent World!')
print("After editing content:", doc.current)

# Undo twice
doc.undo()
print("\nAfter undo:", doc.current)

doc.undo()
print("After another undo:", doc.current)

# Redo
doc.redo()
print("\nAfter redo:", doc.current)

print(f"\n📚 History contains {len(doc.history)} versions")

## 8. Thread Safety

Immutable data structures are naturally thread-safe for reading. Here's a simple example:

In [ ]:
import threading
import time

# Shared immutable configuration
config = PersistentDict.create(
    api_url='https://api.example.com',
    timeout=30,
    retries=3,
    cache_size=1000
)

def worker(worker_id, base_config):
    # Each worker can create its own variant without affecting others
    my_config = base_config.set('worker_id', worker_id)
    my_config = my_config.set('started_at', time.time())
    
    print(f"Worker {worker_id}: {my_config['api_url']} (worker {my_config['worker_id']})")
    return my_config

# Spawn multiple threads - no locks needed!
threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i, config))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("\n✅ All threads completed safely!")
print(f"Original config unchanged: {config}")

## Summary

**pypersistent** provides:

✨ **Four core data structures:**
- `PersistentDict` - Fast hash map (HAMT)
- `PersistentList` - Indexed sequence (Vector Trie)
- `PersistentSet` - Unique elements (HAMT-based)
- `PersistentSortedDict` - Sorted map (Red-Black Tree)

🚀 **Key benefits:**
- 3-5x faster than pure Python alternatives
- 24-3271x faster for creating multiple versions
- Thread-safe by design
- Native pickle support
- Python 3.13+ free-threading ready

📚 **Learn more:**
- GitHub: https://github.com/cmarschner/pypersistent
- PyPI: https://pypi.org/project/pypersistent/
- Documentation: See README on GitHub

## Try It Yourself!

Feel free to experiment with the code above or try your own examples below:

In [ ]:
# Your experiments here!
